# Grafón: Qué electivas me conviene cursar?

Este grafo se encarga de analizar "gente que haya tenido experiencias facultativas similares", sin importar en que año o cuatrimestre fue. La idea es que si me fue parecido en varias materias a alguna otra persona, y esa persona curso alguna electiva que yo no, entonces esa materia es una buena candidata para mí.

Por ejemplo: Por más que Juan se haya recibido el año pasado, es una persona que se parece mucho a mí porque nos sacamos las mismas notas en muchas materias. Probablemente las electivas que Juan eligió, me sirvan a mí como guía.

## Cómo es el grafo?

El grafo analizado va a ser un grafo simple, y no un multigrafo: entre cada par de alumnos solo puede haber una arista. Manejamos la cardinalidad en el peso, en vez de tener múltiples aristas.

- Nodos: alumnos
- Aristas: conectar dos alumnos que hayan cursado misma materia y "les fue parecido".
    - A los dos nos fue muy bien (nos sacamos entre 8 y 10)    
    - A los dos nos fue más o menos (nos sacamos un 6 o un 7)
    - A los dos nos fue mal (nos sacamos un 4 o un 5)
- Peso: La inversa* de la cantidad de materias en donde somos similares

*Calculamos la inversa porque mientras mas similares son, menor peso queremos que haya entre la arista, para que más cercanos esten

### Dos formas de armar el análisis
// ToDo: que hacemos aca? Elegimos uno, o presentamos ambos en el mismo notebook?
// Miti miti? la parte de abajo mas enfocada en solo electivas¨
- sólo correr sobre materias electivas
- correr sobre todas las materias, pero filtrar el output por electivas

In [ ]:
import pandas as pd
import utils

df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
df_simil = utils.curar_data(df)
display(df_simil.sample(3))
df_simil.shape

In [ ]:
df_simil.sort_values('cant_materias_similares', ascending=False).head(15)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.from_pandas_edgelist(df_simil_agg, 
                            source='Padron_x', 
                            target='Padron_y', 
                            edge_attr='inv_cant_materias_similares',
                            create_using=nx.Graph())

print(G)
print(f"""
  El diámetro de la red: {nx.diameter(G)}
  El grado promedio de la red: {sum([n[1] for n in G.degree()]) / len(G):.2f}
  Puentes globales: {list(nx.bridges(G))}
""")

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30,10))
ax = axes.flatten()

# ToDo: Elegir un randomsample (si? no?) para graficar y que se entienda mejor el grafico. Tal vez el grupo del tp puede ser el sample
nx.draw_networkx(G, pos=nx.circular_layout(G), width=0.0005, node_size=50, with_labels=False, ax=ax[0])
nx.draw_networkx(G, pos=nx.spiral_layout(G),   width=0.005,  node_size=50, with_labels=False, ax=ax[1])

## Comunidades

In [ ]:
from networkx.algorithms import community
import matplotlib.colors as mcolors
import random

random.seed(42)
plt.figure(figsize=(30,10))

louvain = community.louvain_communities(G, weight='inv_cant_materias_similares')
draw_nodes = {}
colors = random.sample(list(mcolors.CSS4_COLORS), len(louvain))
for louvaincommunity, color in zip(louvain, colors):
    draw_nodes.update({n: color for n in louvaincommunity})
    
plt.title(f"{len(louvain)} Louvain Communities")
nx.draw_networkx(G, 
                 nodelist=draw_nodes.keys(), 
                 node_color=list(draw_nodes.values()), 
                 width=0.0005, 
                 pos=nx.random_layout(G, seed=42),
                 font_size=10)

In [ ]:
from config import PADRON, CARRERA, plan_estudios

def nestedsearch(el, lst_of_sets):
    return list(filter(lambda lst: el in lst, lst_of_sets))[0]

def materias_padron(padron):
    return df[(df['Padron'] == padron) & (df['materia_nota'] >= 4)]['materia_id'].values

def sugerir_electivas(padron):
    min_alumnos, max_alumnos = 6, 20
    max_iteraciones = 25

    i = 0
    grupo = []
    for i in range(max_iteraciones):
        louvain = community.louvain_communities(G, weight='inv_cant_materias_similares', resolution=1+(i*0.01))
        comunidad = nestedsearch(padron, louvain)
        if min_alumnos <= len(comunidad) <= max_alumnos:
            grupo = comunidad
            break
        elif not grupo or (len(comunidad) >= max_alumnos and (len(comunidad) - max_alumnos <= len(grupo) - max_alumnos)):
            grupo = comunidad
        i+=1
    
    df_sugerencias = df_rel[df_rel['Padron'].isin(grupo)].groupby('materia_id').agg(cant_alumnos_similares=('materia_id', 'count'))
    df_sugerencias = df_sugerencias[~df_sugerencias.index.isin(materias_padron(padron))]
    
    df_materias = pd.read_json(plan_estudios(CARRERA))
    df_sugerencias = pd.merge(df_sugerencias, df_materias, left_on='materia_id', right_on="id")
    df_sugerencias = df_sugerencias[df_sugerencias['categoria'] == 'Materias Electivas']
    df_sugerencias = df_sugerencias[['id', 'materia', 'creditos', 'cant_alumnos_similares']].sort_values('cant_alumnos_similares', ascending=False)
    return df_sugerencias.reset_index(drop=True)

electivas = sugerir_electivas(PADRON)
electivas.head(10)

# Aleatoriedad del grafo

Para esta parte del análisis, vamos a remover todas las aristas que conectan a los alumnos que no sean de electivas. De esta manera, vamos a tener una mejor visibilidad y mejores resultados, ya que el grafo anterior es casi completo, lo cual es difícil de comparar con los modelos vistos en clase.

In [ ]:
df_electivas = utils.curar_data(utils.generar_subdf_materias_electivas(df))

G_electivas = nx.from_pandas_edgelist(df_electivas, 
                            source='Padron_x', 
                            target='Padron_y', 
                            edge_attr='cant_materias_similares',
                            create_using=nx.Graph())

print(G_electivas)
print(f"""
Red FIUBA-Map
  El diámetro de la red: {nx.diameter(G_electivas)}
  El grado promedio de la red: {sum([n[1] for n in G_electivas.degree()]) / len(G_electivas):.2f}
  La distancia promedio de la red: {nx.average_shortest_path_length(G_electivas)}
  Clustering promedio: {"%3.4f"%nx.average_clustering(G_electivas)}
  Puentes globales: {list(nx.bridges(G_electivas))}
  {"Es conexa" if nx.is_connected(G_electivas) else "No es conexa"}
""")

Vamos a realizar una simulación de un modelado de Erdös-Rényi y un modelado de Preferential Attachment (ley de potencias) que correspondan a los parámetros de nuestra red, para luego poder compararlos y poder entender si nuestra red es aleatoria. Para eso vamos a conocer el diametro, grado promedio, distancia promedio, clustering promedio y si se trata de una componente gigante. Al finalizar, vamos realizar una representación de anonymus walks para cada una de las redes y vamos a determinar por distancia coseno cuál sería la simulación más afin y por qué.

In [ ]:
g_erdos = nx.erdos_renyi_graph(G_electivas.number_of_nodes(), 0.6)
print(f"""
Red aleatoria Erdös-Renyi
  El diámetro de la red: {nx.diameter(g_erdos)}
  El grado promedio de la red: {sum([n[1] for n in g_erdos.degree()]) / len(g_erdos):.2f}
  La distancia promedio de la red: {nx.average_shortest_path_length(g_erdos)}
  Clustering promedio: {"%3.4f"%nx.average_clustering(g_erdos)}
  Puentes globales: {list(nx.bridges(g_erdos))}
  {"Es conexa" if nx.is_connected(g_erdos) else "No es conexa"}
""")

In [ ]:
g_preferential_attachment = nx.barabasi_albert_graph(G_electivas.number_of_nodes(), G_electivas.number_of_nodes()//G_electivas.number_of_nodes())
print(f"""
Red aleatoria Preferential Attachment
  El diámetro de la red: {nx.diameter(g_preferential_attachment)}
  El grado promedio de la red: {sum([n[1] for n in g_preferential_attachment.degree()]) / len(g_preferential_attachment):.2f}
  La distancia promedio de la red: {nx.average_shortest_path_length(g_preferential_attachment)}
  Clustering promedio: {nx.average_clustering(g_preferential_attachment)}
  {"Es conexa" if nx.is_connected(g_preferential_attachment) else "No es conexa"}
""")

In [ ]:
aw_erdos = utils.anoymous_walks(g_erdos)
aw_preferential = utils.anoymous_walks(g_preferential_attachment)
aw_original = utils.anoymous_walks(G_electivas)

utils.plot_anoymous_walks([aw_original, aw_erdos, aw_preferential])

In [ ]:
from numpy import linalg as LA

print(f"""
Las leyes de potencias aparecen de la \033[1m ventaja acumulativa\033[0m. Esto puede verse como un desbalance desproporcionado entre los que tienen muchos contactos, y los que tienen pocos. Es claro que el grafon no iba a ser similar a el grafo generado por Barabási–Albert, ya que no tiene sentido que un alumno tenga muchisimas aristas, mientras que otros tengan pocas ya que hay muchas materias en comun. La distancia de coseno entre nuestro grafon y el grafo aleatorio es de {1 - np.inner(aw_original[7],  aw_preferential[7]) / (LA.norm(aw_original[7]) * LA.norm( aw_preferential[7]))}. \n
En cambio, al compararlo con el grafo generado con Erdös Renyi, podemos encontrar más similitudes. A diferencia del anterior, todos los nodos tienen probabilidad de contar con muchas aristas, dejando un grafo más real y parecido a una red de una Universidad en donde todos los alumnos tienden a cursar materias similares. La distancia de coseno entre los mismos {1 - np.inner(aw_original[7],  aw_erdos[7]) / (LA.norm(aw_original[7]) * LA.norm(aw_erdos[7]))}
""")

Es interesante notar, que a pesar de que el grado promedio entre nuestro grafon y Erdös Renyi son similares, la distribución de los mismos no lo es. Como ya sabemos, la representación de Erdös es una campana de Gauss, en donde predomina el grado promedio. Esto podría suceder en una universidad, considerando todas las carreras, pero al tomar una sola carrera y únicamente sus electivas, tiene sentido observar una distribución uniforme, en donde algunos alumnos cursaron muchas materias electivas y cuentan con más aristas y algunos aún cuentan con pocas. 

In [ ]:
utils.plot_distribucion_grados([G_electivas, g_erdos, g_preferential_attachment])